In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import json
import logging

In [2]:
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36",
    "Content-Type": "text/plain;charset=UTF-8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "cs-CZ,cs;q=0.9,en;q=0.8,sk;q=0.7"
}

In [4]:
data = []

In [5]:
def parseArticle(soup, href):
    title = soup.find("h1").text
    author = soup.find("div", {"rel":"rnews:creator schema:creator"}).find("a").text
    published = soup.find("meta", {"itemprop":"datePublished"})["content"]
    modified = soup.find("meta", {"itemprop":"dateModified"})["content"]
    score = soup.find("section", {"class":"field field-name-field-readability-score field-type-list-float field-label-inline clearfix view-mode-_custom_display"}).find("div", {"class":"field-item even"}).text.strip()

    item = {
        'title': title,
        'author': author,
        'link': href,
        'published': published,
        'modified': modified,
        'score': score
    }
    
    #print(item)
    
    data.append(item)

In [6]:
def crawler(seed):
    frontier=[seed]
    crawled=[]
    while frontier:
        page=frontier.pop()
        try:
            if page not in crawled:
                print(page)
                sleep(1)
                source = requests.get(page).text
                soup=BeautifulSoup(source, "html5lib")
                if '/article/' in page:
                    parseArticle(soup, page)
                links=soup.findAll('a',href=True)
                for link in links:
                    new_link = link['href']
                    if(new_link.startswith(seed)):
                        frontier.append(new_link)
                crawled.append(page)
 
        except Exception as e:
            print(e)
    return crawled

In [ ]:
crawler("https://www.sciencenewsforstudents.org/")

In [7]:
data

[]

In [ ]:
with open('out.json', 'w') as outfile:
    json.dump(data, outfile)